In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## ENTSO-E electricity demand data

In [2]:
yr_2016 = pd.read_csv("Total Load - Day Ahead _ Actual_201601010000-201701010000.csv")
yr_2017 = pd.read_csv("Total Load - Day Ahead _ Actual_201701010000-201801010000.csv")
yr_2018 = pd.read_csv("Total Load - Day Ahead _ Actual_201801010000-201901010000.csv")
yr_2019 = pd.read_csv("Total Load - Day Ahead _ Actual_201901010000-202001010000.csv")
yr_2020 = pd.read_csv("Total Load - Day Ahead _ Actual_202001010000-202101010000.csv")
yr_2021 = pd.read_csv("Total Load - Day Ahead _ Actual_202101010000-202201010000.csv")
yr_2022 = pd.read_csv("Total Load - Day Ahead _ Actual_202201010000-202301010000.csv")

demand_dataframes=(yr_2016, yr_2017, yr_2018, yr_2019, yr_2020, yr_2021, yr_2022)

demand_2016_2022=pd.DataFrame()
#Converts dataframes to datetime files and adding them to get one single dataframe.
for dataframe in demand_dataframes:
    dataframe.index=pd.to_datetime(dataframe.iloc[:,0].str.split(" - ").str[0], dayfirst=True)
    demand_2016_2022 = pd.concat([demand_2016_2022, dataframe], ignore_index=False)
    
#deleting unnecessary columns
demand_2016_2022=demand_2016_2022.drop(["Day-ahead Total Load Forecast [MW] - BZN|NO1"], axis=1)
demand_2016_2022=demand_2016_2022.drop(["Time (CET/CEST)"], axis=1)

#24 NaN values, so I replace them using forward fill.
demand_2016_2022["Actual Total Load [MW] - BZN|NO1"].isna().sum()
demand_2016_2022["Actual Total Load [MW] - BZN|NO1"].fillna(method='ffill', inplace=True)
demand_2016_2022

#there are seven too many rows in the dataframe, so I check for duplicates.
duplicate_indices = demand_2016_2022.index.duplicated(keep=False)
#print(demand_2016_2022[duplicate_indices])

#deleting duplicates
demand_2016_2022 = demand_2016_2022[~demand_2016_2022.index.duplicated(keep='last')]
demand_2016_2022

,Actual Total Load [MW] - BZN|NO1
Time (CET/CEST),
2016-01-01 00:00:00,4582.0
2016-01-01 01:00:00,4482.0
2016-01-01 02:00:00,4421.0
2016-01-01 03:00:00,4335.0
2016-01-01 04:00:00,4304.0
...,...
2022-12-31 19:00:00,4824.0
2022-12-31 20:00:00,4650.0
2022-12-31 21:00:00,4573.0


## Temperature and precipitation data
Norsk KlimaServiceSenter (https://seklima.met.no/)

In [3]:
weather=pd.read_csv("Temperature and rain 2016-2022.csv", sep=';')

#deleting the two last columns
weather=weather[:-2]

#checking for NaN values
weather["Lufttemperatur"].isna().sum() #0
weather["Nedbør (1 t)"].isna().sum() #0

#Changes dataframe index to datetime
weather.index=pd.to_datetime(weather.iloc[:,2], dayfirst=True)
weather = weather.rename_axis("Time (CET/CEST)")
weather=weather.drop(["Navn","Stasjon", "Tid(norsk normaltid)"], axis=1)

### Inflation data

In [38]:
inf_dataframe=pd.read_csv("inflasjon2015_2022.csv")
inf_dataframe.index=inf_dataframe["time"]
inf_dataframe = inf_dataframe.rename_axis("Time (CET/CEST)")
inf_dataframe = inf_dataframe.drop(["time"], axis=1)
inf_dataframe.index=pd.to_datetime(inf_dataframe.index)

## Creates one common dataframe with both electricity demand and weather data 

In [41]:
feature_dataframe = pd.merge(weather, demand_2016_2022, on="Time (CET/CEST)")
feature_dataframe = pd.merge(feature_dataframe, inf_dataframe, on="Time (CET/CEST)")

feature_dataframe.to_csv('feature_dataframe.tsv', sep='\t', index=False)